In [33]:
import pandas as pd
import numpy as np
import os

path = []
dataset = []
label = []

for dirname, _, filenames in os.walk('./Cat_Dog'):
    for filename in filenames:
        if '.jpg' in filename:
            file_path = dirname + '/' + filename
            path.append(file_path)
            
            if 'training_set' in file_path:
                dataset.append('train')
            elif 'test_set' in file_path:
                dataset.append('test')
            else:
                dataset.append('N/A')
            
            if 'dogs' in file_path:
                label.append('dog')
            elif 'cat' in file_path:
                label.append('cat')
            else:
                label.append('N/A')


In [34]:
pd.set_option('display.max_colwidth', 300)
data_df = pd.DataFrame({'path' : path, 'dataset' : dataset, 'label' : label})
data_df.head()

,path,dataset,label
0,./Cat_Dog\test_set\test_set\cats/cat.4001.jpg,test,cat
1,./Cat_Dog\test_set\test_set\cats/cat.4002.jpg,test,cat
2,./Cat_Dog\test_set\test_set\cats/cat.4003.jpg,test,cat
3,./Cat_Dog\test_set\test_set\cats/cat.4004.jpg,test,cat
4,./Cat_Dog\test_set\test_set\cats/cat.4005.jpg,test,cat


In [35]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df = data_df[data_df['dataset'] == 'train']
test_df = data_df[data_df['dataset'] == 'test']

tr_df, val_df = train_test_split(train_df, test_size = 0.2, stratify = train_df['label'], random_state = 256)

IMAGE_SIZE = 224
BATCH_SIZE = 64

tr_generator = ImageDataGenerator(rescale = 1/255)
train_flow_gen = tr_generator.flow_from_dataframe(dataframe= train_df,
                                                  x_col = 'path',
                                                  y_col = 'label',
                                                  target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                  class_mode = 'categorical',
                                                  batch_size = BATCH_SIZE,
                                                  shuffle = True)

val_generator = ImageDataGenerator(rescale = 1/255)
valid_flow_gen = val_generator.flow_from_dataframe(dataframe= val_df,
                                                   x_col = 'path',
                                                   y_col = 'label',
                                                   target_size= (IMAGE_SIZE, IMAGE_SIZE),
                                                   class_mode = 'categorical',
                                                   batch_size = BATCH_SIZE)

Found 8005 validated image filenames belonging to 2 classes.
Found 1601 validated image filenames belonging to 2 classes.


In [36]:
next(iter(valid_flow_gen))

(array([[[[0.70980394, 0.64705884, 0.58431375],
          [0.7411765 , 0.6784314 , 0.61960787],
          [0.70980394, 0.64705884, 0.5882353 ],
          ...,
          [0.6784314 , 0.6392157 , 0.6       ],
          [0.6862745 , 0.64705884, 0.60784316],
          [0.68235296, 0.6431373 , 0.6039216 ]],
 
         [[0.49411768, 0.43137258, 0.36862746],
          [0.5529412 , 0.4901961 , 0.43137258],
          [0.6156863 , 0.5529412 , 0.49411768],
          ...,
          [0.6784314 , 0.6392157 , 0.6       ],
          [0.6862745 , 0.64705884, 0.60784316],
          [0.68235296, 0.6431373 , 0.6039216 ]],
 
         [[0.7137255 , 0.6509804 , 0.5882353 ],
          [0.69411767, 0.6313726 , 0.57254905],
          [0.69803923, 0.63529414, 0.5764706 ],
          ...,
          [0.6784314 , 0.6392157 , 0.6       ],
          [0.6862745 , 0.64705884, 0.60784316],
          [0.68235296, 0.6431373 , 0.6039216 ]],
 
         ...,
 
         [[0.5686275 , 0.45882356, 0.3647059 ],
          [0.58823

In [30]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, Sequential

def Create_Model(model_name = 'DG_CT_Similar'):
    input_tensor = Input(shape = (224, 224, 3))
    base_model = VGG16(input_tensor = input_tensor, include_top = False, weights = 'imagenet')
    
    base_md = base_model.output
    
    x = GlobalAveragePooling2D()(base_md)
    x = Dense(50, activation = 'relu')(x)
    output = Dense(2, activation = 'softmax', name = 'Output')(x)
    
    model = Model(inputs = input_tensor, outputs = output)
    model.summary()
    
    return model

In [37]:
model = Create_Model()
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

rlp_cb = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, verbose = 1, patience = 3, mode = 'min')
els_cb = EarlyStopping(monitor = 'val_loss', patience= 10, restore_best_weights= True, mode = 'min', verbose = 1)
# mlc_cb = ModelCheckpoint(filepath = '/{epoch:02d}-{val_loss:.2f}.hdf5', monitor= 'val_loss', patience = 3, save_best_only= True)

history = model.fit_generator(train_flow_gen, epochs = 40, callbacks = [rlp_cb, els_cb], verbose = 1, validation_data = valid_flow_gen)


Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

C:\Users\c\.conda\envs\wongi\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
126/126 [==============================] - 55s 338ms/step - loss: 3.5741 - accuracy: 0.5142 - val_loss: 0.6871 - val_accuracy: 0.5440
Epoch 2/40
126/126 [==============================] - 43s 342ms/step - loss: 0.6909 - accuracy: 0.5309 - val_loss: 0.6712 - val_accuracy: 0.5553
Epoch 3/40
126/126 [==============================] - 41s 328ms/step - loss: 0.6754 - accuracy: 0.5655 - val_loss: 0.6738 - val_accuracy: 0.5047
Epoch 4/40
126/126 [==============================] - 42s 334ms/step - loss: 0.6652 - accuracy: 0.5810 - val_loss: 0.6509 - val_accuracy: 0.6121
Epoch 5/40
126/126 [==============================] - 41s 321ms/step - loss: 0.6362 - accuracy: 0.6215 - val_loss: 0.6203 - val_accuracy: 0.6552
Epoch 6/40
126/126 [==============================] - 40s 316ms/step - loss: 0.6434 - accuracy: 0.6234 - val_loss: 0.6688 - val_accuracy: 0.5878
Epoch 7/40
126/126 [==============================] - 40s 313ms/step - loss: 0.6303 - accuracy: 0.6447 - val_loss: 0.6172 - val_ac

NameError: name 'dg_cat_similar' is not defined

In [38]:
model.save('dg_cat_similar', save_format='h5')